In [1]:
import kagglehub
import pandas as pd
import numpy as np
import tensorflow as tf
from pathlib import Path

# Download latest version
print("📥 Downloading FER2013 dataset...")
path = kagglehub.dataset_download("deadskull7/fer2013")
print(f"✅ Dataset downloaded to: {path}")

def load_fer2013_with_health_labels():
    """Load FER2013 and convert to CLEAR stress/fatigue/anomaly labels"""
    
    # Find the CSV file in the downloaded path
    fer_csv_path = Path(path) / "fer2013.csv"
    
    if not fer_csv_path.exists():
        # Try alternative paths
        possible_files = list(Path(path).rglob("*.csv"))
        if possible_files:
            fer_csv_path = possible_files[0]
            print(f"📁 Found CSV at: {fer_csv_path}")
        else:
            raise FileNotFoundError(f"CSV file not found in {path}")
    
    fer_df = pd.read_csv(fer_csv_path)
    print(f"✅ Loaded FER2013: {len(fer_df)} samples")
    
    # Check the structure of the dataset
    print(f"📋 Columns: {fer_df.columns.tolist()}")
    if 'emotion' not in fer_df.columns:
        print("❌ 'emotion' column not found. Available columns:", fer_df.columns.tolist())
        # Try to find emotion column with different name
        if 'label' in fer_df.columns:
            fer_df = fer_df.rename(columns={'label': 'emotion'})
            print("✅ Renamed 'label' column to 'emotion'")
        else:
            raise KeyError("Could not find emotion/label column in dataset")
    
    print(f"🔢 Emotion value counts:\n{fer_df['emotion'].value_counts().sort_index()}")
    
    # Convert emotion labels to CLEAR binary health labels
    emotion_labels = fer_df['emotion'].values
    
    stress_labels = []
    fatigue_labels = [] 
    anomaly_labels = []
    
    # Emotion mapping for FER2013:
    # 0=Angry, 1=Disgust, 2=Fear, 3=Happy, 4=Sad, 5=Surprise, 6=Neutral
    
    for emotion in emotion_labels:
        # STRESS: Clear binary - negative emotions = stress
        if emotion in [0, 1, 2, 4]:  # Angry, Disgust, Fear, Sad
            stress = 1.0
        else:
            stress = 0.0
            
        # FATIGUE: Clear binary - tired-looking expressions
        if emotion in [4, 6]:  # Sad, Neutral
            fatigue = 1.0
        else:
            fatigue = 0.0
            
        # ANOMALY: Clear binary - rare emotion (Disgust)
        if emotion == 1:  # Disgust (rarest in dataset)
            anomaly = 1.0
        else:
            anomaly = 0.0
            
        stress_labels.append(stress)
        fatigue_labels.append(fatigue)
        anomaly_labels.append(anomaly)
    
    health_labels = {
        'stress': np.array(stress_labels, dtype=np.float32),
        'fatigue': np.array(fatigue_labels, dtype=np.float32),
        'anomaly': np.array(anomaly_labels, dtype=np.float32)
    }
    
    # Process images (resize to 96x96, 3 channels)
    print("🖼️ Processing images...")
    images = []
    
    # Check if we have pixels column
    if 'pixels' not in fer_df.columns:
        # Try to find image data in other columns
        image_columns = [col for col in fer_df.columns if 'pixel' in col.lower() or 'image' in col.lower()]
        if image_columns:
            pixel_col = image_columns[0]
            print(f"✅ Using column '{pixel_col}' for image data")
        else:
            raise KeyError("Could not find pixel data column")
    else:
        pixel_col = 'pixels'
    
    for i, pixel_str in enumerate(fer_df[pixel_col]):
        # Handle different data formats
        if isinstance(pixel_str, str):
            pixels = np.array(pixel_str.split(), dtype=np.float32)
        else:
            pixels = pixel_str.astype(np.float32)
        
        # Reshape to 48x48 (FER2013 standard size)
        try:
            pixels = pixels.reshape(48, 48)
        except:
            # Try other common dimensions
            for size in [48, 64, 96]:
                try:
                    pixels = pixels.reshape(size, size)
                    break
                except:
                    continue
        
        # Convert to 3 channels and resize to 96x96
        img = np.stack([pixels] * 3, axis=-1)
        img = tf.image.resize(img, [96, 96]).numpy()
        images.append(img)
        
        # Progress indicator
        if i % 5000 == 0 and i > 0:
            print(f"   Processed {i}/{len(fer_df)} images...")
    
    images = np.array(images) / 255.0  # Normalize to [0,1]
    
    print(f"📊 Final Dataset:")
    print(f"   Images shape: {images.shape}")
    print(f"   Stress: {np.mean(health_labels['stress']):.1%} positive")
    print(f"   Fatigue: {np.mean(health_labels['fatigue']):.1%} positive") 
    print(f"   Anomaly: {np.mean(health_labels['anomaly']):.1%} positive")
    
    return images, health_labels

# Usage in your training:
print("🚀 Loading FER2013 with health labels...")
X, y = load_fer2013_with_health_labels()

print(f"\n🎯 Dataset ready for training!")
print(f"   X shape: {X.shape}")
print(f"   y keys: {list(y.keys())}")

# Split into train/validation
from sklearn.model_selection import train_test_split

# For multi-output, we need to split indices
indices = np.arange(len(X))
train_idx, val_idx = train_test_split(indices, test_size=0.2, random_state=42, stratify=y['stress'])

X_train, X_val = X[train_idx], X[val_idx]
y_train = {k: v[train_idx] for k, v in y.items()}
y_val = {k: v[val_idx] for k, v in y.items()}

print(f"\n📊 Training set: {len(X_train)} samples")
print(f"📊 Validation set: {len(X_val)} samples")

# Now train your model - it will actually learn!
# model.train(X_train, y_train, X_val, y_val, use_class_weights=True)

2025-11-26 08:43:55.209885: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1764146635.396108      21 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1764146635.446135      21 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

📥 Downloading FER2013 dataset...
✅ Dataset downloaded to: /kaggle/input/fer2013
🚀 Loading FER2013 with health labels...
✅ Loaded FER2013: 35887 samples
📋 Columns: ['emotion', 'pixels', 'Usage']
🔢 Emotion value counts:
emotion
0    4953
1     547
2    5121
3    8989
4    6077
5    4002
6    6198
Name: count, dtype: int64
🖼️ Processing images...


I0000 00:00:1764146654.991660      21 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 15513 MB memory:  -> device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0


   Processed 5000/35887 images...
   Processed 10000/35887 images...
   Processed 15000/35887 images...
   Processed 20000/35887 images...
   Processed 25000/35887 images...
   Processed 30000/35887 images...
   Processed 35000/35887 images...
📊 Final Dataset:
   Images shape: (35887, 96, 96, 3)
   Stress: 46.5% positive
   Fatigue: 34.2% positive
   Anomaly: 1.5% positive

🎯 Dataset ready for training!
   X shape: (35887, 96, 96, 3)
   y keys: ['stress', 'fatigue', 'anomaly']

📊 Training set: 28709 samples
📊 Validation set: 7178 samples


In [2]:
import os
os.makedirs('/kaggle/working/src', exist_ok=True)


In [3]:
%%writefile /kaggle/working/src/train_face_model_kaggle.py
# src/train_face_model_kaggle.py
import tensorflow as tf
import numpy as np
import pandas as pd
from tensorflow.keras import layers, models, callbacks
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight
import json
import logging
import os
from pathlib import Path
import sys

# Add src to path
sys.path.append('/kaggle/working/src')

from stress_model import StressDetectionModel, make_representative_gen_from_numpy

print("✅ TensorFlow version:", tf.__version__)
print("✅ GPU available:", tf.config.list_physical_devices('GPU'))

class FaceModelTrainerKaggle:
    def __init__(self, input_shape=(96, 96, 3)):
        self.input_shape = input_shape
        self.model = None
        self.setup_logging()
        
    def setup_logging(self):
        """Setup logging for Kaggle"""
        logging.basicConfig(
            level=logging.INFO,
            format='%(asctime)s - %(levelname)s - %(message)s'
        )
        self.logger = logging.getLogger('FaceModelTrainer')
    
    def load_fer2013_data(self):
        """Load FER2013 dataset from Kaggle input"""
        self.logger.info("📥 Loading FER2013 dataset from Kaggle...")
        
        try:
            # Kaggle dataset path
            fer_csv = Path("/kaggle/input/fer2013/fer2013.csv")
            fer_df = pd.read_csv(fer_csv)
            self.logger.info(f"📊 Total samples: {len(fer_df):,}")
            
            # Preprocess images
            X, y = [], []
            
            for idx, row in fer_df.iterrows():
                if idx % 5000 == 0:
                    self.logger.info(f"  Processed {idx}/{len(fer_df)} images...")
                
                # Convert pixel string to array
                pixels = np.array(row['pixels'].split(), dtype='float32').reshape(48, 48)
                pixels = np.stack([pixels]*3, axis=-1)  # 3 channels
                pixels /= 255.0  # Normalize
                X.append(pixels)
                y.append(row['emotion'])
            
            X = np.array(X)
            
            # Convert to categorical (7 emotions)
            from tensorflow.keras.utils import to_categorical
            y = to_categorical(y, num_classes=7)
            
            # Resize to target shape
            X_resized = np.array([tf.image.resize(img, self.input_shape[:2]).numpy() for img in X])
            
            # Convert emotions to stress/fatigue labels - FIXED CLEAR BINARY LABELS
            stress_labels, fatigue_labels, anomaly_labels = self._emotion_to_health_labels(y)
            
            labels = {
                'stress': stress_labels,
                'fatigue': fatigue_labels,
                'anomaly': anomaly_labels
            }
            
            self.logger.info(f"✅ Loaded FER2013 data: {X_resized.shape}")
            return X_resized, labels
            
        except Exception as e:
            self.logger.error(f"❌ Error loading data: {e}")
            raise
    
    def _emotion_to_health_labels(self, emotion_labels):
        """Convert FER2013 emotions to CLEAR BINARY stress/fatigue labels"""
        stress_labels = []
        fatigue_labels = []
        anomaly_labels = []
        
        for emotion_vec in emotion_labels:
            emotion_idx = np.argmax(emotion_vec)
            
            # FIXED: CLEAR BINARY LABELS - no overlapping ranges!
            # Stress: 1 for negative emotions, 0 otherwise
            if emotion_idx in [0, 1, 2, 4]:  # Angry, Disgust, Fear, Sad
                stress = 1.0
            else:
                stress = 0.0
            
            # Fatigue: 1 for tired-looking expressions, 0 otherwise
            if emotion_idx in [4, 6]:  # Sad, Neutral
                fatigue = 1.0
            else:
                fatigue = 0.0
            
            # Anomaly: 1 for rare emotion (Disgust), 0 otherwise
            if emotion_idx == 1:  # Disgust (rarest)
                anomaly = 1.0
            else:
                anomaly = 0.0
            
            stress_labels.append(stress)
            fatigue_labels.append(fatigue)
            anomaly_labels.append(anomaly)
        
        return np.array(stress_labels), np.array(fatigue_labels), np.array(anomaly_labels)
    
    def calculate_class_weights(self, y_train):
        """Calculate class weights to handle imbalanced data"""
        self.logger.info("⚖️ Calculating class weights...")
        class_weights = {}
        
        for task in ['stress', 'fatigue', 'anomaly']:
            if task in y_train:
                # Convert to binary labels for weight calculation
                binary_labels = (y_train[task] > 0.5).astype(int)
                
                try:
                    weights = compute_class_weight(
                        'balanced', 
                        classes=np.unique(binary_labels), 
                        y=binary_labels
                    )
                    class_weights[task] = {0: float(weights[0]), 1: float(weights[1])}
                    self.logger.info(f"✅ {task} class weights: {class_weights[task]}")
                except Exception as e:
                    self.logger.warning(f"⚠️ Using default weights for {task}: {e}")
                    class_weights[task] = {0: 1.0, 1: 1.0}
        
        return class_weights
    
    def prepare_data(self):
        """Prepare training data"""
        X, labels = self.load_fer2013_data()
        
        # Split data
        X_train, X_test = train_test_split(X, test_size=0.2, random_state=42)
        
        y_train = {}
        y_test = {}
        for key in labels.keys():
            y_train[key], y_test[key] = train_test_split(
                labels[key], test_size=0.2, random_state=42
            )
        
        self.logger.info(f"📚 Training samples: {len(X_train):,}")
        self.logger.info(f"📚 Validation samples: {len(X_test):,}")
        
        # Log class distribution
        self.logger.info("📊 Class Distribution:")
        for task in ['stress', 'fatigue', 'anomaly']:
            pos_rate = np.mean(y_train[task] > 0.5)
            self.logger.info(f"   {task}: {pos_rate:.1%} positive")
        
        return X_train, X_test, y_train, y_test
    
    def train_model(self, epochs=50, batch_size=32, use_class_weights=True):
        """Train the model on Kaggle GPU with class weights"""
        self.logger.info("🏋️ Starting training on Kaggle GPU...")
        
        # Prepare data
        X_train, X_test, y_train, y_test = self.prepare_data()
        
        # Calculate class weights
        class_weights = None
        if use_class_weights:
            class_weights = self.calculate_class_weights(y_train)
        
        # Build model
        self.model = StressDetectionModel(self.input_shape)
        self.model.build_temporal_attention_model()
        
        # Compile with improved metrics for imbalanced data
        self.model.model.compile(
            optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
            loss={
                'stress': 'binary_crossentropy',
                'fatigue': 'binary_crossentropy',
                'anomaly': 'binary_crossentropy'
            },
            metrics={
                'stress': [
                    tf.keras.metrics.Precision(name='precision'),
                    tf.keras.metrics.Recall(name='recall'),
                    tf.keras.metrics.AUC(name='auc'),  # Better for imbalanced data
                    tf.keras.metrics.BinaryAccuracy(name='accuracy')
                ],
                'fatigue': [
                    tf.keras.metrics.Precision(name='precision'),
                    tf.keras.metrics.Recall(name='recall'),
                    tf.keras.metrics.AUC(name='auc'),
                    tf.keras.metrics.BinaryAccuracy(name='accuracy')
                ],
                'anomaly': [
                    tf.keras.metrics.Precision(name='precision'),
                    tf.keras.metrics.Recall(name='recall'),
                    tf.keras.metrics.AUC(name='auc'),
                    tf.keras.metrics.BinaryAccuracy(name='accuracy')
                ]
            }
        )
        
        # Kaggle-optimized callbacks
        callbacks_list = [
            callbacks.EarlyStopping(
                monitor='val_loss',
                patience=10,
                restore_best_weights=True
            ),
            callbacks.ReduceLROnPlateau(
                monitor='val_loss',
                factor=0.5,
                patience=5,
                min_lr=1e-7
            ),
            callbacks.ModelCheckpoint(
                '/kaggle/working/face_analyzer_best.h5',
                monitor='val_stress_auc',  # Monitor AUC instead of loss
                save_best_only=True,
                mode='max'
            ),
            callbacks.CSVLogger('/kaggle/working/training_log.csv')
        ]
        
        # Train with GPU and class weights
        self.logger.info("🚀 Training started...")
        if class_weights:
            self.logger.info(f"🎯 Using class weights: {class_weights}")
        
        history = self.model.model.fit(
            X_train,
            {
                'stress': y_train['stress'],
                'fatigue': y_train['fatigue'],
                'anomaly': y_train['anomaly']
            },
            batch_size=batch_size,
            epochs=epochs,
            validation_data=(
                X_test,
                {
                    'stress': y_test['stress'],
                    'fatigue': y_test['fatigue'],
                    'anomaly': y_test['anomaly']
                }
            ),
            callbacks=callbacks_list,
            class_weight=class_weights,  # ADDED CLASS WEIGHTS
            verbose=1
        )
        
        # Save final model
        self.model.model.save('/kaggle/working/face_analyzer.h5')
        
        # Save history
        with open('/kaggle/working/training_history.json', 'w') as f:
            json.dump({k: [float(x) for x in v] for k, v in history.history.items()}, f)
        
        # Enhanced evaluation
        self.enhanced_evaluation(X_test, y_test)
        
        self.logger.info("✅ Training complete!")
        return history
    
    def enhanced_evaluation(self, X_test, y_test):
        """Enhanced evaluation with detailed metrics"""
        self.logger.info("📊 Running enhanced evaluation...")
        
        predictions = self.model.model.predict(X_test, verbose=0)
        
        print("\n" + "="*60)
        print("🎯 ENHANCED MODEL EVALUATION")
        print("="*60)
        
        for i, task in enumerate(['stress', 'fatigue', 'anomaly']):
            if task in y_test:
                true_probs = y_test[task]
                pred_probs = predictions[i].flatten()
                true_binary = (true_probs > 0.5).astype(int)
                
                from sklearn.metrics import roc_auc_score, f1_score, classification_report
                
                # Calculate AUC
                try:
                    auc = roc_auc_score(true_binary, pred_probs)
                except:
                    auc = 0.0
                
                # Find optimal threshold
                best_f1 = 0
                best_threshold = 0.5
                
                for threshold in [0.3, 0.4, 0.5, 0.6, 0.7]:
                    pred_binary = (pred_probs > threshold).astype(int)
                    f1 = f1_score(true_binary, pred_binary, zero_division=0)
                    if f1 > best_f1:
                        best_f1 = f1
                        best_threshold = threshold
                
                print(f"\n{task.upper()}:")
                print(f"  AUC: {auc:.4f}")
                print(f"  Best F1: {best_f1:.4f} (threshold: {best_threshold:.2f})")
                print(f"  Positive Rate: {np.mean(true_binary):.1%}")
        
        print("="*60)
    
    def convert_to_tflite(self):
        """Convert to TFLite for edge deployment"""
        if self.model is None:
            raise ValueError("Train model first!")
        
        self.logger.info("🔄 Converting to TFLite...")
        
        # Load some data for quantization
        X_train, _, _, _ = self.prepare_data()
        representative_gen = make_representative_gen_from_numpy(X_train[:100])
        
        # Convert
        self.model.save_as_tflite(
            filepath='/kaggle/working/face_analyzer_int8.tflite',
            representative_gen=representative_gen,
            full_integer=True
        )
        
        self.logger.info("✅ TFLite model saved!")

def main():
    """Main training function for Kaggle"""
    print("🚀 Starting Edge Health Guardian Training on Kaggle")
    print("==================================================")
    
    # Create output directory
    Path('/kaggle/working').mkdir(exist_ok=True)
    
    # Initialize and train
    trainer = FaceModelTrainerKaggle(input_shape=(96, 96, 3))
    
    try:
        # Train model WITH CLASS WEIGHTS
        history = trainer.train_model(
            epochs=30,
            batch_size=64,
            use_class_weights=True  # ENABLE CLASS WEIGHTS
        )
        
        # Convert to TFLite
        trainer.convert_to_tflite()
        
        # List output files
        print("\n📁 Output files created:")
        for file in Path('/kaggle/working').glob('*'):
            if file.is_file():
                size_mb = file.stat().st_size / (1024 * 1024)
                print(f"  {file.name} ({size_mb:.1f} MB)")
        
        print("\n🎉 Training completed successfully!")
        
    except Exception as e:
        print(f"❌ Training failed: {e}")
        import traceback
        traceback.print_exc()

if __name__ == "__main__":
    main()

Writing /kaggle/working/src/train_face_model_kaggle.py


In [4]:
%%writefile /kaggle/working/src/stress_model.py
# stress_model.py
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
import tensorflow_model_optimization as tfmot
from typing import Generator, Dict, List, Union
from pathlib import Path
from sklearn.utils.class_weight import compute_class_weight

class StressDetectionModel:
    def __init__(self, input_shape: tuple):
        self.input_shape = input_shape
        self.model: tf.keras.Model = None
        self.class_weights: Dict = None

    def build_temporal_attention_model(self) -> tf.keras.Model:
        """Build CNN + LSTM + temporal attention for stress, fatigue, anomaly detection"""
        inputs = tf.keras.Input(shape=self.input_shape)

        # Spatial feature extraction (CNN)
        x = layers.Conv2D(32, (3, 3), activation='relu', padding='same')(inputs)
        x = layers.BatchNormalization()(x)
        x = layers.MaxPooling2D((2, 2))(x)

        x = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(x)
        x = layers.BatchNormalization()(x)
        x = layers.MaxPooling2D((2, 2))(x)

        x = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(x)
        x = layers.BatchNormalization()(x)
        x = layers.GlobalAveragePooling2D()(x)

        # Feature transformation
        x = layers.Dense(256, activation='relu')(x)
        x = layers.BatchNormalization()(x)
        x = layers.Dropout(0.3)(x)
        
        x = layers.Dense(128, activation='relu')(x)
        x = layers.BatchNormalization()(x)
        x = layers.Dropout(0.2)(x)
        
        x = layers.Dense(64, activation='relu')(x)
        x = layers.Dropout(0.1)(x)

        # Multi-output for stress, fatigue, anomaly
        stress_output = layers.Dense(1, activation='sigmoid', name='stress')(x)
        fatigue_output = layers.Dense(1, activation='sigmoid', name='fatigue')(x)
        anomaly_output = layers.Dense(1, activation='sigmoid', name='anomaly')(x)

        self.model = models.Model(inputs=inputs, outputs=[stress_output, fatigue_output, anomaly_output])
        return self.model

    def build_temporal_attention_model_complex(self) -> tf.keras.Model:
        """Alternative: Use TFLite-compatible LSTM without CuDNN"""
        inputs = tf.keras.Input(shape=self.input_shape)

        # Spatial feature extraction (CNN)
        x = layers.Conv2D(32, (3, 3), activation='relu', padding='same')(inputs)
        x = layers.BatchNormalization()(x)
        x = layers.MaxPooling2D((2, 2))(x)

        x = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(x)
        x = layers.BatchNormalization()(x)
        x = layers.MaxPooling2D((2, 2))(x)

        x = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(x)
        x = layers.BatchNormalization()(x)
        x = layers.GlobalAveragePooling2D()(x)

        # Create sequence by repeating features (simulate temporal dimension)
        x = layers.RepeatVector(1)(x)  # Single timestep sequence
        
        # Use TFLite-compatible LSTM (disable CuDNN)
        x = layers.LSTM(64, return_sequences=True, implementation=2)(x)  # implementation=2 avoids CuDNN
        
        # Simple attention mechanism (avoid complex operations that break TFLite)
        attention = layers.Dense(1, activation='tanh')(x)
        attention = layers.Flatten()(attention)
        attention = layers.Activation('softmax')(attention)
        
        # Apply attention
        x = layers.Flatten()(x)
        attended_features = layers.Dot(axes=1)([x, attention])
        
        # Fully connected layers
        x = layers.Dense(128, activation='relu')(attended_features)
        x = layers.Dropout(0.3)(x)
        x = layers.Dense(64, activation='relu')(x)
        x = layers.Dropout(0.2)(x)

        # Multi-output for stress, fatigue, anomaly
        stress_output = layers.Dense(1, activation='sigmoid', name='stress')(x)
        fatigue_output = layers.Dense(1, activation='sigmoid', name='fatigue')(x)
        anomaly_output = layers.Dense(1, activation='sigmoid', name='anomaly')(x)

        self.model = models.Model(inputs=inputs, outputs=[stress_output, fatigue_output, anomaly_output])
        return self.model

    def calculate_class_weights(self, train_labels: Dict[str, np.ndarray]):
        """Calculate class weights to handle imbalanced data"""
        self.class_weights = {}
        
        for task in ['stress', 'fatigue', 'anomaly']:
            if task in train_labels:
                labels = train_labels[task]
                # Convert continuous labels to binary for weight calculation
                binary_labels = (labels > 0.5).astype(int)
                
                try:
                    class_weights = compute_class_weight(
                        'balanced', 
                        classes=np.unique(binary_labels), 
                        y=binary_labels
                    )
                    self.class_weights[task] = {
                        0: float(class_weights[0]), 
                        1: float(class_weights[1])
                    }
                    print(f"✅ {task} class weights: {self.class_weights[task]}")
                except Exception as e:
                    print(f"⚠️ Could not calculate weights for {task}: {e}")
                    self.class_weights[task] = {0: 1.0, 1: 1.0}

    def compile_model(self, lr: float = 1e-3, use_improved_metrics: bool = True):
        """Compile model with improved metrics for imbalanced data"""
        if self.model is None:
            raise ValueError("Build the model first (call build_temporal_attention_model).")

        if use_improved_metrics:
            # Use metrics that work better with imbalanced data
            metrics_config = {
                'stress': [
                    tf.keras.metrics.Precision(name='precision'),
                    tf.keras.metrics.Recall(name='recall'),
                    tf.keras.metrics.AUC(name='auc'),  # Better for imbalanced data
                    tf.keras.metrics.BinaryAccuracy(name='accuracy')
                ],
                'fatigue': [
                    tf.keras.metrics.Precision(name='precision'),
                    tf.keras.metrics.Recall(name='recall'), 
                    tf.keras.metrics.AUC(name='auc'),
                    tf.keras.metrics.BinaryAccuracy(name='accuracy')
                ],
                'anomaly': [
                    tf.keras.metrics.Precision(name='precision'),
                    tf.keras.metrics.Recall(name='recall'),
                    tf.keras.metrics.AUC(name='auc'),
                    tf.keras.metrics.BinaryAccuracy(name='accuracy')
                ]
            }
        else:
            # Original metrics
            metrics_config = {
                'stress': [
                    tf.keras.metrics.BinaryAccuracy(name='accuracy'),
                    tf.keras.metrics.Precision(name='precision'),
                    tf.keras.metrics.Recall(name='recall')
                ],
                'fatigue': [
                    tf.keras.metrics.BinaryAccuracy(name='accuracy'),
                    tf.keras.metrics.Precision(name='precision'),
                    tf.keras.metrics.Recall(name='recall')
                ],
                'anomaly': [
                    tf.keras.metrics.BinaryAccuracy(name='accuracy'),
                    tf.keras.metrics.Precision(name='precision'),
                    tf.keras.metrics.Recall(name='recall')
                ]
            }

        self.model.compile(
            optimizer=tf.keras.optimizers.Adam(learning_rate=lr),
            loss={
                'stress': 'binary_crossentropy',
                'fatigue': 'binary_crossentropy',
                'anomaly': 'binary_crossentropy'
            },
            metrics=metrics_config
        )

    def quantize_model(self):
        """Apply quantization-aware training (QAT)"""
        if self.model is None:
            raise ValueError("Build the model first (call build_temporal_attention_model).")
        quantize_model_fn = tfmot.quantization.keras.quantize_model
        self.model = quantize_model_fn(self.model)
        self.compile_model()

    def train(self,
              train_input,
              train_labels,
              val_input,
              val_labels,
              epochs: int = 10,
              batch_size: int = 32,
              callbacks_list = None,
              use_class_weights: bool = True,
              verbose: int = 1):
        """Train the model with optional class weighting"""
        if self.model is None:
            raise ValueError("Build and compile the model before training.")
        
        # Calculate class weights if requested
        if use_class_weights and self.class_weights is None:
            self.calculate_class_weights(train_labels)
        
        # Prepare class weights for training
        class_weight_dict = None
        if use_class_weights and self.class_weights:
            class_weight_dict = self.class_weights
            print("🎯 Using class weights for training:", class_weight_dict)
        
        history = self.model.fit(
            train_input,
            train_labels,
            validation_data=(val_input, val_labels),
            epochs=epochs,
            batch_size=batch_size,
            callbacks=callbacks_list,
            class_weight=class_weight_dict,
            verbose=verbose
        )
        return history

    def evaluate_model(self, test_input, test_labels, threshold: float = 0.5):
        """Enhanced evaluation with custom threshold support"""
        if self.model is None:
            raise ValueError("Model must be trained before evaluation.")
        
        # Standard evaluation
        standard_results = self.model.evaluate(test_input, test_labels, verbose=0)
        
        # Custom evaluation with adjustable threshold
        predictions = self.model.predict(test_input, verbose=0)
        
        print("\n📊 ENHANCED MODEL EVALUATION:")
        print("=" * 50)
        
        for i, task in enumerate(['stress', 'fatigue', 'anomaly']):
            if task in test_labels:
                true_labels = test_labels[task]
                pred_probs = predictions[i].flatten()
                pred_labels = (pred_probs > threshold).astype(int)
                true_binary = (true_labels > 0.5).astype(int)
                
                # Calculate metrics
                from sklearn.metrics import classification_report, f1_score, roc_auc_score
                
                f1 = f1_score(true_binary, pred_labels, zero_division=0)
                try:
                    auc = roc_auc_score(true_binary, pred_probs)
                except:
                    auc = 0.0
                
                print(f"\n{task.upper()} (threshold={threshold}):")
                print(f"  F1-Score: {f1:.4f}")
                print(f"  AUC: {auc:.4f}")
                print(f"  Positive Rate: {np.mean(pred_labels):.4f}")
                print(classification_report(true_binary, pred_labels, 
                                          target_names=['Negative', 'Positive'], 
                                          zero_division=0))
        
        return standard_results

    def save_as_tflite(self,
                       filepath: str,
                       representative_gen: Generator = None,
                       full_integer: bool = True):
        """Convert model to TFLite (optionally INT8 quantized)"""
        if self.model is None:
            raise ValueError("Model must be built/loaded before saving to TFLite.")

        converter = tf.lite.TFLiteConverter.from_keras_model(self.model)
        converter.optimizations = [tf.lite.Optimize.DEFAULT]

        if representative_gen is not None and full_integer:
            converter.representative_dataset = representative_gen
            converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
            converter.inference_input_type = tf.int8
            converter.inference_output_type = tf.int8

        tflite_model = converter.convert()
        Path(filepath).parent.mkdir(parents=True, exist_ok=True)
        with open(filepath, 'wb') as f:
            f.write(tflite_model)
        print(f"✅ TFLite model saved to: {filepath}")

    def find_optimal_threshold(self, val_input, val_labels, task: str = 'stress'):
        """Find optimal classification threshold using validation data"""
        if self.model is None:
            raise ValueError("Model must be trained before threshold optimization.")
        
        predictions = self.model.predict(val_input, verbose=0)
        task_idx = ['stress', 'fatigue', 'anomaly'].index(task)
        pred_probs = predictions[task_idx].flatten()
        true_labels = (val_labels[task] > 0.5).astype(int)
        
        # Find optimal threshold using F1-score
        from sklearn.metrics import f1_score
        best_threshold = 0.5
        best_f1 = 0
        
        for threshold in np.arange(0.1, 0.9, 0.05):
            pred_labels = (pred_probs > threshold).astype(int)
            f1 = f1_score(true_labels, pred_labels, zero_division=0)
            if f1 > best_f1:
                best_f1 = f1
                best_threshold = threshold
        
        print(f"🎯 Optimal threshold for {task}: {best_threshold:.3f} (F1: {best_f1:.3f})")
        return best_threshold

# Representative generator helper
def make_representative_gen_from_numpy(x_samples: np.ndarray, num_steps: int = 100):
    """Yield representative samples for TFLite INT8 quantization"""
    def rep_gen():
        count = 0
        for i in range(min(num_steps, len(x_samples))):
            yield [x_samples[i:i+1].astype(np.float32)]
            count += 1
            if count >= num_steps:
                break
    return rep_gen

# Enhanced training callback for imbalanced data
class ImbalancedDataCallback(tf.keras.callbacks.Callback):
    def __init__(self, validation_data, task_names=['stress', 'fatigue', 'anomaly']):
        super().__init__()
        self.validation_data = validation_data
        self.task_names = task_names
        
    def on_epoch_end(self, epoch, logs=None):
        if epoch % 5 == 0:  # Print every 5 epochs
            print(f"\n📈 Epoch {epoch} - Class Distribution Insights:")
            for task in self.task_names:
                if task in self.validation_data[1]:
                    labels = self.validation_data[1][task]
                    positive_ratio = np.mean(labels > 0.5)
                    print(f"  {task}: {positive_ratio:.1%} positive")

Writing /kaggle/working/src/stress_model.py


In [5]:
%%writefile /kaggle/working/src/train_movement_model_kaggle.py
# train_movement_model_kaggle.py
import tensorflow as tf
import numpy as np
import pandas as pd
from tensorflow.keras import layers, models, callbacks
from sklearn.model_selection import train_test_split
import json
import logging
import os
from pathlib import Path

print("✅ Movement model Kaggle script loaded")

class MovementModelTrainerKaggle:
    def __init__(self, sequence_length=50, feature_dim=12):
        self.sequence_length = sequence_length
        self.feature_dim = feature_dim
        self.model = None
        self.setup_logging()
    
    def setup_logging(self):
        """Setup logging for Kaggle"""
        logging.basicConfig(level=logging.INFO)
        self.logger = logging.getLogger('MovementModelTrainer')
    
    def generate_realistic_movement_data(self, num_samples=15000):
        """Generate realistic movement data simulating accelerometer and gyroscope patterns"""
        self.logger.info("📊 Generating realistic movement data...")
        
        X = []
        stress_labels = []
        fatigue_labels = []
        anomaly_labels = []
        
        for i in range(num_samples):
            # Generate time series data
            time_steps = np.linspace(0, 5, self.sequence_length)
            
            # Base patterns
            if i < num_samples * 0.7:  # 70% normal movement
                # Smooth, coordinated movements
                accel_x = 0.5 * np.sin(2 * np.pi * 1 * time_steps) + 0.1 * np.random.normal(size=self.sequence_length)
                accel_y = 0.5 * np.cos(2 * np.pi * 1 * time_steps) + 0.1 * np.random.normal(size=self.sequence_length)
                accel_z = 9.8 + 0.05 * np.random.normal(size=self.sequence_length)  # Gravity
                
                gyro_x = 0.1 * np.sin(2 * np.pi * 2 * time_steps) + 0.02 * np.random.normal(size=self.sequence_length)
                gyro_y = 0.1 * np.cos(2 * np.pi * 2 * time_steps) + 0.02 * np.random.normal(size=self.sequence_length)
                gyro_z = 0.01 * np.random.normal(size=self.sequence_length)
                
                stress = np.random.uniform(0.0, 0.4)
                fatigue = np.random.uniform(0.0, 0.3)
                anomaly = 0.0
                
            elif i < num_samples * 0.85:  # 15% stressed movement
                # Jerky, tense movements with tremors
                tremor_freq = 8  # 8Hz tremor
                tremor = 0.2 * np.sin(2 * np.pi * tremor_freq * time_steps)
                
                accel_x = 0.7 * np.sin(2 * np.pi * 1 * time_steps) + tremor + 0.2 * np.random.normal(size=self.sequence_length)
                accel_y = 0.7 * np.cos(2 * np.pi * 1 * time_steps) + tremor + 0.2 * np.random.normal(size=self.sequence_length)
                accel_z = 9.8 + 0.1 * np.random.normal(size=self.sequence_length)
                
                gyro_x = 0.2 * np.sin(2 * np.pi * 2 * time_steps) + 0.1 * tremor + 0.05 * np.random.normal(size=self.sequence_length)
                gyro_y = 0.2 * np.cos(2 * np.pi * 2 * time_steps) + 0.1 * tremor + 0.05 * np.random.normal(size=self.sequence_length)
                gyro_z = 0.05 * np.random.normal(size=self.sequence_length)
                
                stress = np.random.uniform(0.6, 1.0)
                fatigue = np.random.uniform(0.2, 0.6)
                anomaly = 0.0
                
            else:  # 15% fatigued movement
                # Slow, uncoordinated movements
                accel_x = 0.3 * np.sin(2 * np.pi * 0.5 * time_steps) + 0.15 * np.random.normal(size=self.sequence_length)
                accel_y = 0.3 * np.cos(2 * np.pi * 0.5 * time_steps) + 0.15 * np.random.normal(size=self.sequence_length)
                accel_z = 9.8 + 0.2 * np.random.normal(size=self.sequence_length)
                
                gyro_x = 0.05 * np.sin(2 * np.pi * 1 * time_steps) + 0.08 * np.random.normal(size=self.sequence_length)
                gyro_y = 0.05 * np.cos(2 * np.pi * 1 * time_steps) + 0.08 * np.random.normal(size=self.sequence_length)
                gyro_z = 0.08 * np.random.normal(size=self.sequence_length)
                
                stress = np.random.uniform(0.2, 0.5)
                fatigue = np.random.uniform(0.7, 1.0)
                anomaly = 0.0
            
            # Add occasional anomalies (1%)
            if np.random.random() < 0.01:
                anomaly = 1.0
                # Add abnormal patterns
                accel_x += 2.0 * np.random.normal(size=self.sequence_length)
                accel_y += 2.0 * np.random.normal(size=self.sequence_length)
            
            # Combine features
            features = np.column_stack([
                accel_x, accel_y, accel_z,
                gyro_x, gyro_y, gyro_z,
                np.gradient(accel_x),  # Jerk features
                np.gradient(accel_y),
                np.convolve(accel_x, np.ones(5)/5, mode='same'),  # Smoothed
                np.convolve(accel_y, np.ones(5)/5, mode='same'),
                np.sqrt(accel_x**2 + accel_y**2 + accel_z**2),  # Magnitude
                np.sqrt(gyro_x**2 + gyro_y**2 + gyro_z**2)
            ])
            
            X.append(features)
            stress_labels.append(stress)
            fatigue_labels.append(fatigue)
            anomaly_labels.append(anomaly)
        
        X = np.array(X)
        labels = {
            'stress': np.array(stress_labels),
            'fatigue': np.array(fatigue_labels),
            'anomaly': np.array(anomaly_labels)
        }
        
        self.logger.info(f"✅ Generated {num_samples} movement samples")
        return X, labels
    
    def build_movement_model(self):
        """Build LSTM-based movement analysis model"""
        inputs = tf.keras.Input(shape=(self.sequence_length, self.feature_dim))
        
        # Bidirectional LSTM for temporal patterns
       
        x = layers.Bidirectional(layers.LSTM(64, return_sequences=True, implementation=2))(inputs)

        x = layers.BatchNormalization()(x)
        x = layers.Dropout(0.3)(x)
        
        x = layers.Bidirectional(layers.LSTM(32, return_sequences=False, implementation=2))(x)
        x = layers.BatchNormalization()(x)
        x = layers.Dropout(0.2)(x)
        
        # Attention mechanism
        # (For simplicity, using dense layers instead of proper attention for sequences)
        attention_weights = layers.Dense(64, activation='tanh')(x)
        attention_weights = layers.Dense(1, activation='sigmoid')(attention_weights)
        
        # Apply attention (simplified)
        attended = layers.multiply([x, attention_weights])
        
        # Fully connected layers
        x = layers.Dense(128, activation='relu')(attended)
        x = layers.Dropout(0.2)(x)
        x = layers.Dense(64, activation='relu')(x)
        x = layers.Dropout(0.1)(x)
        
        # Multi-output
        stress_output = layers.Dense(1, activation='sigmoid', name='stress')(x)
        fatigue_output = layers.Dense(1, activation='sigmoid', name='fatigue')(x)
        anomaly_output = layers.Dense(1, activation='sigmoid', name='anomaly')(x)
        
        self.model = models.Model(inputs=inputs, outputs=[stress_output, fatigue_output, anomaly_output])
        return self.model
    
    def train_model(self, epochs=20, batch_size=128):
        """Train movement model on Kaggle GPU"""
        self.logger.info("🏋️ Training movement analysis model...")
        
        # Generate data
        X, labels = self.generate_realistic_movement_data(15000)
        
        # Split data
        X_train, X_test = train_test_split(X, test_size=0.2, random_state=42)
        y_train = {k: v[:len(X_train)] for k, v in labels.items()}
        y_test = {k: v[len(X_train):] for k, v in labels.items()}
        
        # Build model
        self.build_movement_model()
        
        # Compile
        self.model.compile(
            optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
            loss={
                'stress': 'binary_crossentropy',
                'fatigue': 'binary_crossentropy', 
                'anomaly': 'binary_crossentropy'
            },
            metrics=['accuracy', 'precision', 'recall']
        )
        
        # Callbacks
        callbacks_list = [
            callbacks.EarlyStopping(
                monitor='val_loss',
                patience=8,
                restore_best_weights=True
            ),
            callbacks.ReduceLROnPlateau(
                monitor='val_loss',
                factor=0.5,
                patience=5,
                min_lr=1e-7
            ),
            callbacks.ModelCheckpoint(
                '/kaggle/working/movement_analyzer_best.h5',
                monitor='val_stress_loss',
                save_best_only=True
            )
        ]
        
        # Train
        self.logger.info("Starting training...")
        history = self.model.fit(
            X_train, y_train,
            validation_data=(X_test, y_test),
            epochs=epochs,
            batch_size=batch_size,
            callbacks=callbacks_list,
            verbose=1
        )
        
        # Save final model
        self.model.save('/kaggle/working/movement_analyzer.h5')
        
        # Save history
        with open('/kaggle/working/movement_training_history.json', 'w') as f:
            json.dump({k: [float(x) for x in v] for k, v in history.history.items()}, f)
        
        self.logger.info("✅ Movement model training complete!")
        return history
    
    def convert_to_tflite(self):
        """Convert to TFLite format"""
        if self.model is None:
            raise ValueError("Train the model first!")
        
        self.logger.info("🔄 Converting movement model to TFLite...")
        
        # Load some data for representative dataset
        X, _ = self.generate_realistic_movement_data(100)
        
        def representative_gen():
            for i in range(50):
                yield [X[i:i+1].astype(np.float32)]
        
        # Convert
        converter = tf.lite.TFLiteConverter.from_keras_model(self.model)
        converter.optimizations = [tf.lite.Optimize.DEFAULT]
        converter.representative_dataset = representative_gen
        converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
        converter.inference_input_type = tf.int8
        converter.inference_output_type = tf.int8
        
        tflite_model = converter.convert()
        
        with open('/kaggle/working/movement_analyzer_int8.tflite', 'wb') as f:
            f.write(tflite_model)
        
        self.logger.info("✅ Movement TFLite model saved!")

def main():
    """Main function for movement model training"""
    print("🚀 Starting Movement Model Training on Kaggle")
    
    # Initialize and train
    trainer = MovementModelTrainerKaggle(sequence_length=50, feature_dim=12)
    
    try:
        # Train model
        history = trainer.train_model(epochs=20, batch_size=128)
        
        # Convert to TFLite
        trainer.convert_to_tflite()
        
        print("\n🎉 Movement model training completed successfully!")
        
    except Exception as e:
        print(f"❌ Movement model training failed: {e}")
        import traceback
        traceback.print_exc()

if __name__ == "__main__":
    main()

Writing /kaggle/working/src/train_movement_model_kaggle.py


In [6]:
%%writefile /kaggle/working/src/train_fusion_model_kaggle.py 
# train_fusion_model_kaggle.py
import tensorflow as tf
import numpy as np
import pandas as pd
from tensorflow.keras import layers, models, callbacks
from sklearn.model_selection import train_test_split
import json
import logging
import os
from pathlib import Path

print("✅ Fusion model Kaggle script loaded")

class FusionModelTrainerKaggle:
    def __init__(self):
        self.model = None
        self.setup_logging()
    
    def setup_logging(self):
        """Setup logging for Kaggle"""
        logging.basicConfig(level=logging.INFO)
        self.logger = logging.getLogger('FusionModelTrainer')
    
    def generate_fusion_data(self, num_samples=10000):
        """Generate realistic fusion data combining face, movement, and HR features"""
        self.logger.info("📊 Generating fusion training data...")
        
        # Simulate outputs from individual models
        face_features = np.random.normal(0, 1, (num_samples, 32)).astype(np.float32)
        movement_features = np.random.normal(0, 1, (num_samples, 24)).astype(np.float32)
        hr_features = np.random.normal(0, 1, (num_samples, 3)).astype(np.float32)
        
        # Generate realistic final labels based on combined features
        stress_labels = []
        fatigue_labels = []
        anomaly_labels = []
        
        for i in range(num_samples):
            # Combine information from all sensors
            face_stress_indicator = np.mean(face_features[i, :8])  # First 8 face features
            movement_stress_indicator = np.mean(movement_features[i, 6:12])  # Movement stress features
            hr_stress_indicator = hr_features[i, 2]  # HR variability
            
            # Final stress score (weighted combination)
            stress_score = (
                0.4 * self._sigmoid(face_stress_indicator) +
                0.4 * self._sigmoid(movement_stress_indicator) + 
                0.2 * hr_stress_indicator +
                np.random.normal(0, 0.1)
            )
            stress_labels.append(np.clip(stress_score, 0, 1))
            
            # Fatigue calculation
            face_fatigue_indicator = np.mean(face_features[i, 8:16])  # Face fatigue features
            movement_fatigue_indicator = np.mean(movement_features[i, :6])  # Movement smoothness
            
            fatigue_score = (
                0.5 * self._sigmoid(face_fatigue_indicator) +
                0.5 * self._sigmoid(movement_fatigue_indicator) +
                np.random.normal(0, 0.1)
            )
            fatigue_labels.append(np.clip(fatigue_score, 0, 1))
            
            # Anomaly detection (based on feature inconsistencies)
            feature_consistency = np.std([face_stress_indicator, movement_stress_indicator, hr_stress_indicator])
            anomaly_score = min(1.0, feature_consistency * 2.0)
            
            # Add rare true anomalies
            if np.random.random() < 0.02:  # 2% true anomalies
                anomaly_labels.append(1.0)
            else:
                anomaly_labels.append(1.0 if anomaly_score > 0.8 and np.random.random() < 0.3 else 0.0)
        
        inputs = {
            'face_features': face_features,
            'movement_features': movement_features,
            'hr_features': hr_features
        }
        
        labels = {
            'stress': np.array(stress_labels),
            'fatigue': np.array(fatigue_labels),
            'anomaly': np.array(anomaly_labels)
        }
        
        self.logger.info(f"✅ Generated {num_samples} fusion samples")
        return inputs, labels
    
    def _sigmoid(self, x):
        """Helper sigmoid function"""
        return 1 / (1 + np.exp(-x))
    
    def build_fusion_model(self):
        """Build sensor fusion model with attention mechanism"""
        # Inputs from different sensors
        face_input = tf.keras.Input(shape=(32,), name='face_features')
        movement_input = tf.keras.Input(shape=(24,), name='movement_features')
        hr_input = tf.keras.Input(shape=(3,), name='hr_features')
        
        # Feature transformation with batch normalization
        face_branch = layers.Dense(64, activation='relu')(face_input)
        face_branch = layers.BatchNormalization()(face_branch)
        face_branch = layers.Dropout(0.2)(face_branch)
        
        movement_branch = layers.Dense(64, activation='relu')(movement_input)
        movement_branch = layers.BatchNormalization()(movement_branch)
        movement_branch = layers.Dropout(0.2)(movement_branch)
        
        hr_branch = layers.Dense(16, activation='relu')(hr_input)
        hr_branch = layers.BatchNormalization()(hr_branch)
        hr_branch = layers.Dropout(0.2)(hr_branch)
        
        # Concatenate all features
        concatenated = layers.concatenate([face_branch, movement_branch, hr_branch])
        
        # Attention mechanism for feature weighting
        attention = layers.Dense(concatenated.shape[-1], activation='tanh')(concatenated)
        attention = layers.Dense(concatenated.shape[-1], activation='softmax')(attention)
        
        # Apply attention
        attended_features = layers.multiply([concatenated, attention])
        
        # Main fusion network
        x = layers.Dense(128, activation='relu')(attended_features)
        x = layers.BatchNormalization()(x)
        x = layers.Dropout(0.3)(x)
        
        x = layers.Dense(64, activation='relu')(x)
        x = layers.BatchNormalization()(x)
        x = layers.Dropout(0.2)(x)
        
        x = layers.Dense(32, activation='relu')(x)
        
        # Multi-output for final health assessment
        stress_output = layers.Dense(1, activation='sigmoid', name='stress')(x)
        fatigue_output = layers.Dense(1, activation='sigmoid', name='fatigue')(x)
        anomaly_output = layers.Dense(1, activation='sigmoid', name='anomaly')(x)
        
        self.model = models.Model(
            inputs=[face_input, movement_input, hr_input],
            outputs=[stress_output, fatigue_output, anomaly_output]
        )
        
        return self.model
    
    def train_model(self, epochs=15, batch_size=64):
        """Train fusion model on Kaggle GPU"""
        self.logger.info("🏋️ Training sensor fusion model...")
        
        # Generate data
        inputs, labels = self.generate_fusion_data(10000)
        
        # Build model
        self.build_fusion_model()
        
        # Compile
        self.model.compile(
            optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
            loss={
                'stress': 'binary_crossentropy',
                'fatigue': 'binary_crossentropy',
                'anomaly': 'binary_crossentropy'
            },
            metrics=['accuracy', 'precision', 'recall']
        )
        
        # Split data
        train_idx, test_idx = train_test_split(range(len(inputs['face_features'])), test_size=0.2, random_state=42)
        
        X_train = {k: v[train_idx] for k, v in inputs.items()}
        X_test = {k: v[test_idx] for k, v in inputs.items()}
        y_train = {k: v[train_idx] for k, v in labels.items()}
        y_test = {k: v[test_idx] for k, v in labels.items()}
        
        # Callbacks
        callbacks_list = [
            callbacks.EarlyStopping(
                monitor='val_loss',
                patience=8,
                restore_best_weights=True
            ),
            callbacks.ReduceLROnPlateau(
                monitor='val_loss',
                factor=0.5,
                patience=4,
                min_lr=1e-7
            ),
            callbacks.ModelCheckpoint(
                '/kaggle/working/fusion_engine_best.h5',
                monitor='val_stress_loss',
                save_best_only=True
            )
        ]
        
        # Train
        self.logger.info("Starting training...")
        history = self.model.fit(
            X_train, y_train,
            validation_data=(X_test, y_test),
            epochs=epochs,
            batch_size=batch_size,
            callbacks=callbacks_list,
            verbose=1
        )
        
        # Save final model
        self.model.save('/kaggle/working/fusion_engine.h5')
        
        # Save history
        with open('/kaggle/working/fusion_training_history.json', 'w') as f:
            json.dump({k: [float(x) for x in v] for k, v in history.history.items()}, f)
        
        self.logger.info("✅ Fusion model training complete!")
        return history
    
    def convert_to_tflite(self):
        """Convert to TFLite format"""
        if self.model is None:
            raise ValueError("Train the model first!")
        
        self.logger.info("🔄 Converting fusion model to TFLite...")
        
        # Generate representative data
        inputs, _ = self.generate_fusion_data(100)
        
        def representative_gen():
            for i in range(50):
                yield {
                    'face_features': inputs['face_features'][i:i+1].astype(np.float32),
                    'movement_features': inputs['movement_features'][i:i+1].astype(np.float32),
                    'hr_features': inputs['hr_features'][i:i+1].astype(np.float32)
                }
        
        # Convert
        converter = tf.lite.TFLiteConverter.from_keras_model(self.model)
        converter.optimizations = [tf.lite.Optimize.DEFAULT]
        converter.representative_dataset = representative_gen
        converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
        converter.inference_input_type = tf.int8
        converter.inference_output_type = tf.int8
        
        tflite_model = converter.convert()
        
        with open('/kaggle/working/fusion_engine_int8.tflite', 'wb') as f:
            f.write(tflite_model)
        
        self.logger.info("✅ Fusion TFLite model saved!")

def main():
    """Main function for fusion model training"""
    print("🚀 Starting Fusion Model Training on Kaggle")
    
    # Initialize and train
    trainer = FusionModelTrainerKaggle()
    
    try:
        # Train model
        history = trainer.train_model(epochs=15, batch_size=64)
        
        # Convert to TFLite
        trainer.convert_to_tflite()
        
        print("\n🎉 Fusion model training completed successfully!")
        
    except Exception as e:
        print(f"❌ Fusion model training failed: {e}")
        import traceback
        traceback.print_exc()

if __name__ == "__main__":
    main()

Writing /kaggle/working/src/train_fusion_model_kaggle.py


In [7]:

print("🚀 Setting up Edge Health Guardian Training on Kaggle")
!pip install tensorflow-model-optimization



🚀 Setting up Edge Health Guardian Training on Kaggle
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.5/242.5 kB 5.5 MB/s eta 0:00:00


In [8]:
# Verify the fix is there
with open('/kaggle/working/src/train_movement_model_kaggle.py', 'r') as f:
    content = f.read()
    if 'implementation=2' in content:
        print("✅ Movement model fix confirmed!")
    else:
        print("❌ Movement model fix missing!")

✅ Movement model fix confirmed!


In [9]:
%%writefile /kaggle/working/src/train_all_models_kaggle.py
import os
import sys
from pathlib import Path

print("🚀 Training ALL Edge Health Guardian Models on Kaggle")

# Add to path
sys.path.append('/kaggle/working/src')

# Import training scripts
from train_face_model_kaggle import FaceModelTrainerKaggle
from train_movement_model_kaggle import MovementModelTrainerKaggle
from train_fusion_model_kaggle import FusionModelTrainerKaggle

def train_all_models():
    """Train all three models sequentially"""
    
    print("=" * 60)
    print("🎯 Starting Complete Model Training")
    print("=" * 60)
    
    # 1. Train Face Model
    print("\n1️⃣  Training Face Analysis Model...")
    face_trainer = FaceModelTrainerKaggle()
    face_trainer.train_model(epochs=30, batch_size=64)
    face_trainer.convert_to_tflite()
    
    # 2. Train Movement Model  
    print("\n2️⃣  Training Movement Analysis Model...")
    movement_trainer = MovementModelTrainerKaggle()
    movement_trainer.train_model(epochs=20, batch_size=128)
    movement_trainer.convert_to_tflite()
    
    # 3. Train Fusion Model
    print("\n3️⃣  Training Sensor Fusion Model...")
    fusion_trainer = FusionModelTrainerKaggle()
    fusion_trainer.train_model(epochs=15, batch_size=64)
    fusion_trainer.convert_to_tflite()
    
    print("\n🎉 ALL MODELS TRAINED SUCCESSFULLY!")

if __name__ == "__main__":
    train_all_models()

Writing /kaggle/working/src/train_all_models_kaggle.py


In [10]:
from train_face_model_kaggle import *
print(dir())


ModuleNotFoundError: No module named 'train_face_model_kaggle'

In [ ]:
# src/train_all_models_kaggle.py
import os
import sys
from pathlib import Path
import numpy as np
from sklearn.utils.class_weight import compute_class_weight

print("🚀 Training ALL Edge Health Guardian Models on Kaggle")

# Add to path
sys.path.append('/kaggle/working/src')

# Import training scripts
from train_face_model_kaggle import FaceModelTrainerKaggle
from train_movement_model_kaggle import MovementModelTrainerKaggle
from train_fusion_model_kaggle import FusionModelTrainerKaggle

def calculate_class_weights_for_face_model(y_train):
    """Calculate class weights for face model tasks"""
    class_weights = {}
    
    for task in ['stress', 'fatigue', 'anomaly']:
        if task in y_train:
            # Convert to binary labels for weight calculation
            binary_labels = (y_train[task] > 0.5).astype(int)
            
            try:
                weights = compute_class_weight(
                    'balanced', 
                    classes=np.unique(binary_labels), 
                    y=binary_labels
                )
                class_weights[task] = {0: float(weights[0]), 1: float(weights[1])}
                print(f"✅ {task} class weights: {class_weights[task]}")
            except Exception as e:
                print(f"⚠️ Using default weights for {task}: {e}")
                class_weights[task] = {0: 1.0, 1: 1.0}
    
    return class_weights

def train_all_models():
    """Train all three models sequentially with improved settings"""
    
    print("=" * 60)
    print("🎯 Starting Complete Model Training")
    print("=" * 60)
    
    # 1. Train Face Model
    print("\n1️⃣  Training Face Analysis Model...")
    face_trainer = FaceModelTrainerKaggle()
    
    # Load data first to calculate class weights
    print("📥 Loading face data for class weight calculation...")
    from train_face_model_kaggle import load_fer2013_with_health_labels
    X_face, y_face = load_fer2013_with_health_labels()
    
    # Split data
    from sklearn.model_selection import train_test_split
    indices = np.arange(len(X_face))
    train_idx, val_idx = train_test_split(indices, test_size=0.2, random_state=42)
    
    X_train = X_face[train_idx]
    X_val = X_face[val_idx]
    y_train = {k: v[train_idx] for k, v in y_face.items()}
    y_val = {k: v[val_idx] for k, v in y_face.items()}
    
    # Calculate class weights
    class_weights = calculate_class_weights_for_face_model(y_train)
    
    # Train with class weights
    face_trainer.train_model_with_data(
        X_train=X_train, y_train=y_train, 
        X_val=X_val, y_val=y_val,
        class_weights=class_weights,
        epochs=30, 
        batch_size=64
    )
    face_trainer.convert_to_tflite()
    
    # 2. Train Movement Model  
    print("\n2️⃣  Training Movement Analysis Model...")
    movement_trainer = MovementModelTrainerKaggle()
    movement_trainer.train_model(epochs=20, batch_size=128)
    movement_trainer.convert_to_tflite()
    
    # 3. Train Fusion Model
    print("\n3️⃣  Training Sensor Fusion Model...")
    fusion_trainer = FusionModelTrainerKaggle()
    fusion_trainer.train_model(epochs=15, batch_size=64)
    fusion_trainer.convert_to_tflite()
    
    print("\n🎉 ALL MODELS TRAINED SUCCESSFULLY!")

if __name__ == "__main__":
    train_all_models()

In [ ]:
# src/train_movement_model_kaggle.py
import tensorflow as tf
import numpy as np
from sklearn.model_selection import train_test_split
import logging
from pathlib import Path

print("✅ Movement model script loaded")

class MovementModelTrainerKaggle:
    def __init__(self, sequence_length=50, feature_dim=12):
        self.sequence_length = sequence_length
        self.feature_dim = feature_dim
        self.model = None
        self.setup_logging()
    
    def setup_logging(self):
        logging.basicConfig(level=logging.INFO)
        self.logger = logging.getLogger('MovementModelTrainer')
    
    def generate_synthetic_movement_data(self, num_samples=10000):
        """Generate synthetic movement data"""
        self.logger.info("📊 Generating movement data...")
        
        # Simulate accelerometer and gyroscope data
        X = np.random.randn(num_samples, self.sequence_length, self.feature_dim).astype(np.float32)
        
        # Generate labels
        stress_labels = np.random.uniform(0, 1, num_samples)
        fatigue_labels = np.random.uniform(0, 1, num_samples) 
        anomaly_labels = (np.random.random(num_samples) > 0.95).astype(float)
        
        labels = {
            'stress': stress_labels,
            'fatigue': fatigue_labels,
            'anomaly': anomaly_labels
        }
        
        return X, labels
    
    def train_model(self, epochs=20, batch_size=128):
        """Train movement model on Kaggle"""
        self.logger.info("🏋️ Training movement model...")
        
        # Generate data
        X, labels = self.generate_synthetic_movement_data()
        
        # Split data
        X_train, X_test = train_test_split(X, test_size=0.2, random_state=42)
        y_train = {k: v[:len(X_train)] for k, v in labels.items()}
        y_test = {k: v[len(X_train):] for k, v in labels.items()}
        
        # Build simple LSTM model (you can replace with your actual architecture)
        from tensorflow.keras import layers, models
        
        inputs = tf.keras.Input(shape=(self.sequence_length, self.feature_dim))
        x = layers.LSTM(64, return_sequences=True)(inputs)
        x = layers.LSTM(32)(x)
        x = layers.Dense(64, activation='relu')(x)
        x = layers.Dropout(0.3)(x)
        
        stress_output = layers.Dense(1, activation='sigmoid', name='stress')(x)
        fatigue_output = layers.Dense(1, activation='sigmoid', name='fatigue')(x)
        anomaly_output = layers.Dense(1, activation='sigmoid', name='anomaly')(x)
        
        self.model = models.Model(inputs=inputs, outputs=[stress_output, fatigue_output, anomaly_output])
        
        # Compile
        self.model.compile(
            optimizer='adam',
            loss={'stress': 'mse', 'fatigue': 'mse', 'anomaly': 'binary_crossentropy'},
            metrics=['mae']
        )
        
        # Train
        history = self.model.fit(
            X_train, y_train,
            validation_data=(X_test, y_test),
            epochs=epochs,
            batch_size=batch_size,
            verbose=1
        )
        
        # Save
        self.model.save('/kaggle/working/movement_analyzer.h5')
        self.logger.info("✅ Movement model saved!")
        
        return history
    
    def convert_to_tflite(self):
        """Convert to TFLite"""
        if self.model is None:
            raise ValueError("Train model first!")
        
        converter = tf.lite.TFLiteConverter.from_keras_model(self.model)
        converter.optimizations = [tf.lite.Optimize.DEFAULT]
        tflite_model = converter.convert()
        
        with open('/kaggle/working/movement_analyzer_int8.tflite', 'wb') as f:
            f.write(tflite_model)
        
        self.logger.info("✅ Movement TFLite model saved!")

In [ ]:
# src/train_fusion_model_kaggle.py
import tensorflow as tf
import numpy as np
from sklearn.model_selection import train_test_split
import logging

print("✅ Fusion model script loaded")

class FusionModelTrainerKaggle:
    def __init__(self):
        self.model = None
        self.setup_logging()
    
    def setup_logging(self):
        logging.basicConfig(level=logging.INFO)
        self.logger = logging.getLogger('FusionModelTrainer')
    
    def generate_fusion_data(self, num_samples=5000):
        """Generate synthetic fusion data"""
        self.logger.info("📊 Generating fusion data...")
        
        # Simulate outputs from face and movement models
        face_features = np.random.randn(num_samples, 32).astype(np.float32)
        movement_features = np.random.randn(num_samples, 24).astype(np.float32)
        hr_features = np.random.randn(num_samples, 3).astype(np.float32)
        
        # Generate final labels
        stress_labels = np.random.uniform(0, 1, num_samples)
        fatigue_labels = np.random.uniform(0, 1, num_samples)
        anomaly_labels = (np.random.random(num_samples) > 0.98).astype(float)
        
        inputs = {
            'face': face_features,
            'movement': movement_features, 
            'hr': hr_features
        }
        
        labels = {
            'stress': stress_labels,
            'fatigue': fatigue_labels,
            'anomaly': anomaly_labels
        }
        
        return inputs, labels
    
    def train_model(self, epochs=15, batch_size=64):
        """Train fusion model"""
        self.logger.info("🏋️ Training fusion model...")
        
        # Generate data
        inputs, labels = self.generate_fusion_data()
        
        # Build fusion model
        from tensorflow.keras import layers, models
        
        face_input = tf.keras.Input(shape=(32,), name='face_features')
        movement_input = tf.keras.Input(shape=(24,), name='movement_features') 
        hr_input = tf.keras.Input(shape=(3,), name='hr_features')
        
        # Process each input
        face_branch = layers.Dense(64, activation='relu')(face_input)
        movement_branch = layers.Dense(64, activation='relu')(movement_input)
        hr_branch = layers.Dense(16, activation='relu')(hr_input)
        
        # Concatenate
        concatenated = layers.concatenate([face_branch, movement_branch, hr_branch])
        
        # Final layers
        x = layers.Dense(128, activation='relu')(concatenated)
        x = layers.Dropout(0.3)(x)
        x = layers.Dense(64, activation='relu')(x)
        
        # Outputs
        stress_output = layers.Dense(1, activation='sigmoid', name='stress')(x)
        fatigue_output = layers.Dense(1, activation='sigmoid', name='fatigue')(x)
        anomaly_output = layers.Dense(1, activation='sigmoid', name='anomaly')(x)
        
        self.model = models.Model(
            inputs=[face_input, movement_input, hr_input],
            outputs=[stress_output, fatigue_output, anomaly_output]
        )
        
        # Compile
        self.model.compile(
            optimizer='adam',
            loss={'stress': 'mse', 'fatigue': 'mse', 'anomaly': 'binary_crossentropy'},
            metrics=['mae']
        )
        
        # Split data
        train_idx, test_idx = train_test_split(range(len(inputs['face'])), test_size=0.2)
        
        X_train = {k: v[train_idx] for k, v in inputs.items()}
        X_test = {k: v[test_idx] for k, v in inputs.items()}
        y_train = {k: v[train_idx] for k, v in labels.items()}
        y_test = {k: v[test_idx] for k, v in labels.items()}
        
        # Train
        history = self.model.fit(
            X_train, y_train,
            validation_data=(X_test, y_test),
            epochs=epochs,
            batch_size=batch_size,
            verbose=1
        )
        
        # Save
        self.model.save('/kaggle/working/fusion_engine.h5')
        self.logger.info("✅ Fusion model saved!")
        
        return history
    
    def convert_to_tflite(self):
        """Convert to TFLite"""
        if self.model is None:
            raise ValueError("Train model first!")
        
        converter = tf.lite.TFLiteConverter.from_keras_model(self.model)
        converter.optimizations = [tf.lite.Optimize.DEFAULT]
        tflite_model = converter.convert()
        
        with open('/kaggle/working/fusion_engine_int8.tflite', 'wb') as f:
            f.write(tflite_model)
        
        self.logger.info("✅ Fusion TFLite model saved!")